In [135]:
import pandas as pd
import requests

def get_ranking(ranking_type='headphones'):
    url = f'https://crinacle.com/rankings/{ranking_type}/'

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers=header)
    dat = pd.read_html(r.text)[0]
    
    # Deal with both missing prices and weirdly formatted prices
    dat = dat.query('Pricesort != 99999')
    dat['Price (MSRP)'] = pd.to_numeric(dat['Price (MSRP)'], errors='coerce')
    dat.dropna(subset=['Price (MSRP)'], inplace=True)
    
    # Add the bang for buck variable
    ranks = dat['Rank'].unique()
    rank_number = np.linspace(100, 1, len(ranks))
    rank_number_lut = dict(zip(ranks, rank_number))
    dat['Rank_number'] = dat['Rank'].replace(rank_number_lut)
    dat['bang_buck'] = (dat['Rank_number'] / dat['Price (MSRP)'])
    
    dat.attrs['rank_number_lut'] = rank_number_lut
    return dat

def select_products(dat, min_price = 50, max_price = 100, min_rank='C+', sort_to='bang_buck', open_closed=None):
    subset = (
        dat
          .query(f'{max_price} >= `Price (MSRP)` >= {min_price}')                   # Price Range
          .query('Rank_number >= %f' % dat.attrs['rank_number_lut'][min_rank])    # Minimum Rank
          .sort_values(sort_to, ascending=False)             # Sort according to bang for buck
    )
    if open_closed is not None:
        subset[subset['Fit/Cup Type'].str.contains(open_closed)]
    return subset

headphone_data = get_ranking('iems')

<ipython-input-135-e59c8be250d0>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['Price (MSRP)'] = pd.to_numeric(dat['Price (MSRP)'], errors='coerce')
<ipython-input-135-e59c8be250d0>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat.dropna(subset=['Price (MSRP)'], inplace=True)
<ipython-input-135-e59c8be250d0>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [142]:
select_products(headphone_data, min_rank='B', 
                min_price=20, max_price=100).head(10)

,Rank,Value Rating,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Setup,Based on,Note weight,Ranksort,Tonesort,Techsort,Pricesort,Rank_number,bang_buck
235,B,NaN,Moondrop Crescent,30.0,Neutral with bass boost,Well-executed V-ish signature that's neither h...,A,C+,DD,Third party unit,NaN,112,21,91,30,62.875,2.095833
165,B,★★★,Moondrop Aria,80.0,Harman-neutral,A cheaper Starfield.,A-,B,DD,Review unit,NaN,88,31,57,80,62.875,0.785937
236,B,★★★,Lypertek PurePlay Z3 2.0(PurePlay Z3)(Tevi),90.0,DF-neutral with bass boost,Probably the closest thing to an ER2XR in TWS ...,A,C+,DD,Jaben Melbourne demo unit,NaN,112,21,91,90,62.875,0.698611
